# 🎓 2026 MenaML: Distillation Workshop
- Connect to the T4 GPU!


## 1. Introduction: What is Knowledge Distillation?

**Knowledge Distillation** is a technique whereby a student neural network learns from another, usually already pre-trained neural network. KD can be used for compressing the model, in which case the student is smaller. It can also be used for improving model performance, where the student is the same or even larger than the teacher. In either case, the the student is trained to mimic the behavior of one or more teacher models.

### Why do we need it?

- **Deployment constraints**: Large models are expensive to run on edge devices, mobile phones, or in real-time applications
- **Inference speed**: Smaller models are faster
- **Cost reduction**: Less compute = less money and energy
- **Increased performance**: Sometimes KD is used to increase the performance of our model.




## Learning Objectives

By the end of this workshop, you will:
1. Understand the core concepts behind Distillation
2. Implement a Teacher-Student training framework from scratch
3. Compare different training strategies: baseline, hard labels, soft labels
4. Analyze the trade-offs between model size and performance

---

## Why Does KD work?

### The "Dark Knowledge"

In their seminal 2015 paper, [Hinton et al](https://arxiv.org/abs/1503.02531). observed that the **soft probability outputs** of a teacher model contain more information than hard labels.

**Example**: For a cat image, hard label says `[0, 0, 1, 0, ...]` (just "cat")  
But soft labels might say `[0.01, 0.05, 0.85, 0.09, ...]` revealing that the image also looks a bit like a dog or tiger!

This extra information about class relationships is the "dark knowledge" that helps the student learn better.

![Distillation Diagram](https://intellabs.github.io/distiller/imgs/knowledge_distillation.png)


### Reweighing Training Examples

Even if we ignore probabilities for all classes other than the true class, we get an effect where some examples are weighted higher than other ones according to what probability the teacher assigns to the true class. [Tang et al](https://arxiv.org/abs/2002.03532) showed that this *importance weighing* is an important component of how KD works.

## 2. Setup and Imports

In [ ]:
import collections
import os
import time
from typing import Dict, List, Optional, Tuple, Union

# Third party
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision
import torchvision.transforms as transforms
from torchvision import models
import tqdm.auto as tqdm
import random

# Check GPU availability
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

In [ ]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
set_seed(10)

## 3. Dataset: CIFAR-10

We'll use CIFAR-10: 60,000 32x32 color images in 10 classes.

In [ ]:
# Data augmentation and normalization
cifar_mean = np.asarray([0.4914, 0.4822, 0.4465])
cifar_std = np.asarray([0.2023, 0.1994, 0.2010])
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=cifar_mean, std=cifar_std),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=cifar_mean, std=cifar_std),
])

# Download and load datasets
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)

trainloader = DataLoader(
    trainset, batch_size=128, shuffle=True, num_workers=0,
)
testloader = DataLoader(
    testset, batch_size=128, shuffle=False, num_workers=0,
)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

print(f"Training samples: {len(trainset)}")
print(f"Test samples: {len(testset)}")

In [ ]:
# Visualize some samples
def imshow(ax, img: torch.Tensor) -> None:
    """Helper function to un-normalize and display an image.

    Args:
        ax: Matplotlib axes to plot on.
        img (torch.Tensor): Tensor image of shape (C, H, W).

    Returns:
        None
    """
    img = img * cifar_std[:, None, None] + cifar_mean[:, None, None]
    img = np.clip(img, 0., 1.)
    npimg = img.numpy()
    ax.imshow(np.transpose(npimg, (1, 2, 0)))

# Get some random training images
dataiter = iter(trainloader)
images, labels = next(dataiter)

# Show images
fig, axes = plt.subplots(1, 8, figsize=(6, 1))
for i in range(8):
    # axes[i].imshow(np.transpose((images[i] / 2 + 0.5).numpy(), (1, 2, 0)))
    imshow(axes[i], images[i])
    axes[i].set_title(classes[labels[i]])
    axes[i].axis('off')
plt.tight_layout()
plt.show()

## 4. Our Models: Convolutional Neural Networks (CNNs)

Convolution is a linear operation where you multiply pixels from a neighbourhood with a filter and sum the results. This way you get a number for the center of the filter. You apply the convolution to an image by sliding the filter along the image. This is ilustrated in this animation.
![convolution](https://d29g4g2dyqv443.cloudfront.net/sites/default/files/pictures/2018/convolution-2.gif)

A convolutional layer applies many such filter to an image in parallel. You can build a convolutional neural network by stacking such layers one after the other, like in the image below.

![CNN](https://cdn.analyticsvidhya.com/wp-content/uploads/2020/10/90650dnn2.webp)

The models we use here are CNNs. The teacher model is large, with 11M parameters, and it is a standard architecture called ResNet-18. A ResNet is similar to a CNN, but it adds skip-connections between layers.

![ResNet](https://cdn.prod.website-files.com/680a070c3b99253410dd3df5/684d85026cf20a8dfe9fa8fd_6835cb463baf4440d0a5d1c5_Resnet50_fig1.webp)

Our student is a small custom CNN with under 1M parameters.

The goal is to transfer knowledge from the large teacher to the tiny student.

In [ ]:
class TeacherCNN(nn.Module):
    """Teacher: ResNet-18 adapted for CIFAR-10 (32x32 images).

    Args:
      num_classes: the number of outputs in the last layer. For Cifar-10 we use
                   10 outputs.
      weights: The name of the weights. With None, the weights will be randomly
               initialized. You can try 'IMAGENET1K_V1' to load weights trained
               on the ImageNet1K dataset. These should be much stronger than our
               provided weights.
    """
    def __init__(self, num_classes: int = 10, weights: Optional[str] = None):
        super(TeacherCNN, self).__init__()
        self.model = models.resnet18(weights=weights)
        # Modify first conv layer for 32x32 images (no aggressive downsampling)
        self.model.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.model.maxpool = nn.Identity()  # Remove maxpool for small images
        self.fc = nn.Linear(self.model.fc.in_features, num_classes)
        self.model.fc = nn.Identity()

    def forward(self, x, output_features=False):
        x = self.model(x)
        logits = self.fc(x)
        if output_features:
            return logits, x
        return logits


class StudentCNN(nn.Module):
    """
    Student: A small CNN with ~100K parameters
    """
    def __init__(self, num_classes: int = 10, output_dim: int = 256):
        super(StudentCNN, self).__init__()
        self.features = nn.Sequential(
            # Block 1
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),  # 32 -> 16

            # Block 2
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),  # 16 -> 8

            # Block 3
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),  # 8 -> 4

            # Block 4
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),  # 4 -> 2

            # Block 5
            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),  # 2 -> 1
        )

        self.fc1 = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(512 * 1 * 1, output_dim),  # 512 to match teacher
            nn.ReLU(),
        )
        self.fc2 = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(output_dim, num_classes),
        )

    def forward(self, x, output_features=False):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        logits = self.fc2(x)
        if output_features:
            return logits, x
        return logits


def count_parameters(model):
    """Count trainable parameters"""
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

### Activity 1. Instantiate the Teacher and Student models

In [ ]:
# TODO
teacher = None
student = None

#### Solution

In [ ]:
# @title
# Create models and compare sizes
teacher = TeacherCNN().to(device)
student = StudentCNN().to(device)

### Compare Student and Teacher Model Sizes

In [ ]:
teacher_params = count_parameters(teacher)
student_params = count_parameters(student)

print(f"Teacher (ResNet-18) parameters: {teacher_params:,}")
print(f"Student (Small CNN) parameters: {student_params:,}")
print(f"\nCompression ratio: {teacher_params / student_params:.1f}x smaller")

## 5. Training Utilities

In [ ]:
def train_epoch(model, teacher, trainloader, criterion, optimizer, device):
    """Trains the model using supervised training or distillation.

    Distillation is used only when the teacher is present. In that case, the
    teacher is frozen (no gradients).
    """
    model.train()
    if teacher is not None:
      teacher.eval()  # Teacher is always in eval mode

    running_metrics = collections.defaultdict(int)
    correct = 0
    total = 0

    for inputs, labels in tqdm.tqdm(trainloader, desc="Training", leave=False):
        inputs, labels = inputs.to(device), labels.to(device)

        # Forward pass for the trained model
        optimizer.zero_grad()
        logits = model(inputs)

        # Compute loss
        if teacher is not None:
            # Get teacher predictions (no gradient needed)
            with torch.no_grad():
                teacher_logits = teacher(inputs)
            loss, metrics = criterion(
                logits, teacher_logits, labels
            )
        else:
          loss = criterion(logits, labels)
          metrics = {}

        # Backward and optimize
        loss.backward()
        optimizer.step()

        # Metrics
        running_metrics['loss'] += loss.item()
        for k, v in metrics.items():
            running_metrics[k] += v
        _, predicted = logits.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    n = len(trainloader)
    results = {k: v/n for k, v in running_metrics.items()}
    results['train_acc'] = 100. * correct / total
    return results


def evaluate(model, testloader, device):
    """Evaluate model on test set"""
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in testloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

    return dict(test_acc=100. * correct / total)


def train_loop(
    model: nn.Module,
    criterion: nn.Module,
    checkpoint_name: str,
    model_name: str,
    experiment_name: str,
    train_data_loader: DataLoader,
    test_data_loader: DataLoader,
    teacher: Optional[nn.Module] = None,
    num_epochs: int = 5,
    learning_rate: float = 0.01,
    overwrite: bool = False,
    custom_train_epoch_fn = None,
) -> dict[str, list[float]]:
  """Main training loop.

  Args:
      model: Model to train.
      criterion: Loss function.
      checkpoint_name: Filename to save checkpoint.
      model_name: Display name for the model.
      experiment_name: Display name for the experiment.
      train_data_loader: Training loader.
      test_data_loader: Test loader.
      teacher: Optional teacher model.
      num_epochs: Number of epochs.
      learning_rate: Learning rate.
      overwrite: Whether to overwrite existing checkpoints.

  Returns:
      dict[str, list[float]]: Training history.
  """
  # Setup optimizer and scheduler
  optimizer = optim.Adam(model.parameters(), lr=learning_rate)
  scheduler = optim.lr_scheduler.CosineAnnealingLR(
      optimizer, T_max=num_epochs, eta_min=0.001
  )

  print(experiment_name)
  print("=" * 60)
  history = collections.defaultdict(list)
  checkpoint_file_name = f'{checkpoint_name}.pth'

  # Check if checkpoint exists safely
  if os.path.exists(checkpoint_file_name) and not overwrite:
    raise ValueError(f'The checkpoint {checkpoint_file_name} already exists!')

  best_test_acc = 0.0
  for epoch in range(num_epochs):
      train_start = time.time()

      # Run training epoch
      if custom_train_epoch_fn is None:
        metrics = train_epoch(
            model, teacher, train_data_loader, criterion, optimizer, device
        )
      else:
        metrics = custom_train_epoch_fn(
            model, teacher, train_data_loader, criterion, optimizer, device
        )
      train_time = time.time() - train_start

      # Evaluation
      eval_start = time.time()
      metrics.update(evaluate(model, test_data_loader, device))
      eval_time = time.time() - eval_start

      # Update learning rate
      scheduler.step()

      # Store metrics
      for k, v in metrics.items():
          history[k].append(v)

      train_acc = metrics.pop('train_acc')
      test_acc = metrics.pop('test_acc')

      # Log progress
      print(
                f"Epoch {epoch+1:2d}/{num_epochs}",
                ' |'.join(
                    f" {k}: {v:.4f}"
                    for k, v in metrics.items()
                    if teacher is not None or k not in ['hard_loss', 'soft_loss']
                ),
                f"| Train Acc {train_acc:.2f}% | Test Acc {test_acc:.2f}% "
                f"| Epoch time {train_time:.2f}s | Eval time {eval_time:.2f}s"
            )

      # Save best model
      if test_acc > best_test_acc:
          best_test_acc = test_acc
          torch.save(model.state_dict(), checkpoint_file_name)

  print(f"\n✅ {model_name} final test accuracy: {best_test_acc:.2f}%")
  return history


def to_cpu(tensor_or_list):
  """Converts GPU values to CPU ones. Useful for visualisation."""
  return torch.tensor(tensor_or_list).cpu()

## 6. Train the Teacher Model

First, we need a well-trained teacher. In practice, you might use a pre-trained model, but we'll train one from scratch for educational purposes.

**Note**: To save time, we'll provide the checkpoint, but you can check the training code and train your own teacher later.

### 6.1. Download the Teacher Model

In [ ]:
!gdown --id 1Ko41G-TVerBr1tY0cSr4m1h1s9PvHRXw

teacher.load_state_dict(torch.load('teacher_resnet18.pth'))
teacher.eval()

### 6.2. Train the Teacher Model

In [ ]:
# # Training configuration
# TEACHER_EPOCHS = 15  # Increase for better teacher (50+ recommended)
# LEARNING_RATE = 0.01

# teacher = TeacherCNN().to(device)
# teacher_history = train_loop(
#     model=teacher,
#     criterion=nn.CrossEntropyLoss(),
#     train_data_loader=trainloader,
#     test_data_loader=testloader,
#     num_epochs=TEACHER_EPOCHS,
#     learning_rate=LEARNING_RATE,
#     checkpoint_name="teacher_resnet18.pth",
#     model_name="Teacher",
#     experiment_name="Teacher Model (ResNet-18)",
#     overwrite=False,

# )

## 7. Baseline. Student without distillation

Let's start by training the student network with only hard labels. This will allow us to appreciate the value of distillation.

### Activity 2. Train Student WITHOUT Distillation

In [ ]:
#@title Training Hypers
STUDENT_EPOCHS = 5
LR = 0.01

In [ ]:
# Train student from scratch (no teacher)
teacher_dim = teacher.fc.in_features
student_baseline = StudentCNN(output_dim=teacher_dim).to(device)

baseline_history = train_loop(
    model=student_baseline,
    criterion=nn.CrossEntropyLoss(),
    train_data_loader=trainloader,
    test_data_loader=testloader,
    num_epochs=STUDENT_EPOCHS,
    learning_rate=LR,
    checkpoint_name="baseline_student_v1",
    model_name="Baseline Student",
    experiment_name="Training Student Baseline (NO distillation)",
    overwrite=False,
    # overwrite=True,
)

### Solution (Download the student's checkpoint)

In [ ]:
# @title
!gdown --id 1xTYieLn5jpIb9SWplvMqrU3otCQC81Q-

student_baseline.load_state_dict(torch.load('baseline_student_v1.pth'))

## 8. The Core: Knowledge Distillation Loss

### The Distillation Loss Function
The key innovation is combining two losses:
$$L_{total} = \alpha \cdot L_{hard} + (1 - \alpha) \cdot L_{soft}$$
Where:
- $L_{hard}$ = Cross-entropy with true labels (standard supervised loss)
- $L_{soft}$ = KL divergence between teacher and student soft predictions
- $\alpha$ = Weight balancing the two losses (typically 0.1-0.5)

### Activity 3. Implement the `DistillationLoss` class in accordance with the previously provided description.



In [ ]:
class DistillationLoss(nn.Module):
    """Knowledge Distillation Loss.

    Combines:
    1. Hard label loss: classification cross entropy.
    2. Soft label loss: KL divergence between the student and teacher logits.

    The T^2 factor compensates for the gradient magnitude reduction when using
    temperature.
    """
    def __init__(self, alpha: float):
        super().__init__()
        # TODO

    def forward(
          self,
          student_logits: torch.Tensor,
          teacher_logits: torch.Tensor,
          labels: torch.Tensor
        ) -> Tuple[torch.Tensor, torch.Tensor, torch.Tensor]:
        # TODO
        metrics = dict(hard_loss=0., soft_loss=0.)
        return total_loss, metrics

#### Solution

In [ ]:
# @title
class DistillationLoss(nn.Module):
    """Knowledge Distillation Loss.

    Combines:
    1. Hard label loss: classification cross entropy.
    2. Soft label loss: KL divergence between the student and teacher logits.

    The T^2 factor compensates for the gradient magnitude reduction when using
    temperature.
    """
    def __init__(self, alpha: float):
        super().__init__()
        self.alpha = alpha
        self.ce_loss = nn.CrossEntropyLoss()
        self.kl_loss = nn.KLDivLoss(reduction='batchmean')

    def forward(
          self,
          student_logits: torch.Tensor,
          teacher_logits: torch.Tensor,
          labels: torch.Tensor
        ) -> Tuple[torch.Tensor, torch.Tensor, torch.Tensor]:
        # Hard label loss (standard cross-entropy)
        hard_loss = self.ce_loss(student_logits, labels)

        # Soft label loss (KL divergence)
        # Student: log-softmax
        student_logits = F.log_softmax(student_logits, dim=1)
        # Teacher: softmax (target distribution)
        teacher_probs = F.softmax(teacher_logits, dim=1)

        # KL divergence * T^2 (to match gradient magnitude)
        soft_loss = self.kl_loss(student_logits, teacher_probs)

        # Combined loss
        total_loss = self.alpha * hard_loss + (1 - self.alpha) * soft_loss

        metrics = dict(hard_loss=hard_loss, soft_loss=soft_loss)
        return total_loss, metrics

## 9. Train Student with Distillation

### Training with distillation (Using big network as teacher)

#### Activity 4. Implement the training code that uses the distillation loss defined above to distill knowledge from a large pretrained teacher model to a smaller student model.

In [ ]:
# Distillation configuration
STUDENT_EPOCHS = 5
ALPHA = 0.1        # Weight for hard loss (try: 0.1, 0.3, 0.5)
LR = 0.01

# Load trained teacher
# TODO

# Load student
# TODO


# Define training loss
# TODO

# Start the model training.
# TODO

#### Solution

In [ ]:
# @title
# Distillation configuration
STUDENT_EPOCHS = 5
ALPHA = 0.1        # Weight for hard loss (try: 0.1, 0.3, 0.5)
LR = 0.01

# Load trained teacher
teacher.load_state_dict(torch.load('teacher_resnet18.pth'))
teacher.eval()

# Load student
teacher_dim = teacher.fc.in_features
student_distilled = StudentCNN(output_dim=teacher_dim).to(device)

# Define training loss
distill_criterion = DistillationLoss(alpha=ALPHA)

# Start the model training.
distilled_history = train_loop(
    model=student_distilled,
    teacher=teacher,
    criterion=distill_criterion,
    train_data_loader=trainloader,
    test_data_loader=testloader,
    num_epochs=STUDENT_EPOCHS,
    learning_rate=LR,
    checkpoint_name="student_distilled_v1",
    model_name="Distilled Student",
    experiment_name="Training Student with Knowledge Distillation",
    overwrite=False,
    # overwrite=True,
)

## 10. Results Comparison

In [ ]:
# Final comparison
teacher_acc = evaluate(teacher, testloader, device)['test_acc']
distilled_acc = evaluate(student_distilled, testloader, device)['test_acc']
baseline_acc = evaluate(student_baseline, testloader, device)['test_acc']

print("\n" + "="*60)
print("📊 FINAL RESULTS")
print("="*60)
print(f"{'Model':<25} {'Parameters':<15} {'Test Accuracy':<15}")
print("-"*60)
print(f"{'Teacher (ResNet-18)':<25} {teacher_params:>12,} {teacher_acc:>12.2f}%")
print(f"{'Student (Baseline)':<25} {student_params:>12,} {baseline_acc:>12.2f}%")
print(f"{'Student (Distilled)':<25} {student_params:>12,} {distilled_acc:>12.2f}%")
print("-"*60)
print(f"\n📈 Distillation improvement: +{distilled_acc - baseline_acc:.2f}%")
print(f"🗜️  Compression ratio: {teacher_params / student_params:.1f}x fewer parameters")

In [ ]:
#@title Plot training curves
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

xs = np.arange(1, STUDENT_EPOCHS + 1)
# Test accuracy comparison
axes[0].plot(xs, baseline_history['test_acc'], label='Student (Baseline)', linestyle='--')
axes[0].plot(xs, distilled_history['test_acc'], label='Student (Distilled)', linewidth=2)
axes[0].axhline(y=teacher_acc, color='r', linestyle=':', label=f'Teacher ({teacher_acc:.1f}%)')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Test Accuracy (%)')
axes[0].set_title('Test Accuracy Comparison')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Loss breakdown for distilled student
axes[1].plot(xs, to_cpu(distilled_history['hard_loss']), label='Hard Loss (CE)', alpha=0.8)
axes[1].plot(xs, to_cpu(distilled_history['soft_loss']), label='Soft Loss (KL)', alpha=0.8)
axes[1].plot(xs, to_cpu(distilled_history['loss']), label='Total Loss', linewidth=2)
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Loss')
axes[1].set_title('Distillation Loss Components')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 11. Inference Speed Comparison

In [ ]:
def benchmark_inference(
    model: nn.Module,
    input_size: tuple = (1, 3, 32, 32),
    num_iterations: int = 1000
) -> float:
    """Benchmark inference speed.

    Args:
        model (nn.Module): Model to benchmark.
        input_size (tuple): Input tensor shape.
        num_iterations (int): Number of iterations to average.

    Returns:
        float: Average time per inference in milliseconds.
    """
    model.eval()
    dummy_input = torch.randn(input_size).to(device)

    # Warmup
    for _ in range(100):
        with torch.no_grad():
            _ = model(dummy_input)

    # Benchmark
    if device.type == 'cuda':
        torch.cuda.synchronize()

    start = time.time()
    for _ in range(num_iterations):
        with torch.no_grad():
            _ = model(dummy_input)

    if device.type == 'cuda':
        torch.cuda.synchronize()

    elapsed = time.time() - start
    return elapsed / num_iterations * 1000  # ms per inference

teacher_time = benchmark_inference(teacher)
student_time = benchmark_inference(student_distilled)

print("\n⚡ Inference Speed (single sample)")
print("-" * 40)
print(f"Teacher (ResNet-18): {teacher_time:.3f} ms")
print(f"Student (Distilled): {student_time:.3f} ms")
print(f"Speedup: {teacher_time / student_time:.2f}x faster")

## 12. Temperature Scaling
To transfer more of the dark knowledge from the teacher to the student, we "soften" the probability distributions using temperature $T$ applied to the logits $z$:
$$p_i = \frac{\exp(z_i / T)}{\sum_j \exp(z_j / T)}$$
- $T = 1$: Normal softmax
- $T > 1$: Softer distribution (reveals more information about class relationships)
- Typically $T \in [3, 20]$
- The temperature is applied to both the student and the teacher logits.

### 🔍 Let's Visualize Temperature Effects

Understanding how temperature affects the probability distribution is crucial.

In [ ]:
# Demonstrate temperature effect
def visualize_temperature_effect(logits: torch.Tensor, temperatures: list = [1, 2, 4, 8, 16]) -> None:
    """Visualize how temperature affects the softmax distribution.

    Args:
        logits (torch.Tensor): Output logits from a model (1D tensor).
        temperatures (list): List of temperatures to visualize.

    Returns:
        None
    """
    fig, axes = plt.subplots(1, len(temperatures), figsize=(15, 3))

    for ax, T in zip(axes, temperatures):
        probs = F.softmax(logits / T, dim=0).numpy()
        ax.bar(range(len(probs)), probs, color='steelblue')
        ax.set_title(f'T = {T}')
        ax.set_xlabel('Class')
        ax.set_ylabel('Probability')
        ax.set_ylim(0, 1)
        ax.set_xticks(range(10))

    plt.suptitle('Effect of Temperature on Softmax Distribution', fontsize=14)
    plt.tight_layout()
    plt.show()

# Example logits (model is quite confident about class 3)
example_logits = torch.tensor([1.0, 2.0, 1.5, 8.0, 0.5, 2.5, 1.0, 0.8, 1.2, 1.8])
visualize_temperature_effect(example_logits)

print("\n📊 Observation:")
print("- T=1: Sharp distribution (almost one-hot)")
print("- T>1: Softer distribution revealing class relationships")
print("- Higher T = more 'dark knowledge' transferred")

### Activity 5: How do loss gradients change with the temperature?
Remember that the temperature is applied to both the student and the teacher predictions.
Your task:
1. Take the example teacher and student logits below.
2. Write a function to compute the loss and the gradient with respect to the student logits. Hint: the loss is computed in the DistillationLoss class; for gradients you need to do `tensor.backward()` and then you can access the gradient by `tensor.grad`
3. Apply temperature to both the student and the teacher logits.
4. Comptute loss and gradient values for different levels of temperature. How does the gradient change?

Is the changing gradient a problem? What can you do about it?

In [ ]:
# Example student and teacher logits
student_logits = torch.from_numpy(np.asarray([
    -0.21406781, -1.47030687, -0.52430741,  0.88407076,  3.20583727,
    0.27593067, -0.54877487,  0.9942204 ,  0.11794449,  0.63376627
]))

teacher_logits = torch.from_numpy(np.asarray([
    0.64185687,  1.52115532, -0.7592614 , -2.06392082,  0.49512868,
    0.74697671, -0.19236562,  1.91342173,  0.78049914,  0.00639535
]))

In [ ]:
def compute_kl_gradient(student_logits: torch.Tensor, teacher_logits: torch.Tensor, temperature: float = 1.0):
    # Ensure student_logits requires gradients
    student_logits.requires_grad_(True)

    # TODO


    # Return the gradients of student_logits
    return grad

### Solution

In [ ]:
# @title
def compute_kl_gradient(student_logits: torch.Tensor, teacher_logits: torch.Tensor, temperature: float = 1.0):
    # Ensure student_logits requires gradients
    student_logits.requires_grad_(True)

    # Apply temperature to logits
    student_tempered_logits = student_logits / temperature
    teacher_tempered_logits = teacher_logits / temperature

    # Compute log-softmax for student and softmax for teacher
    student_log_probs = F.log_softmax(student_tempered_logits, dim=0)
    teacher_probs = F.softmax(teacher_tempered_logits, dim=0)

    # Compute KL divergence loss
    loss = nn.KLDivLoss(reduce='mean')(student_log_probs, teacher_probs)

    # Compute gradients
    loss.backward()

    # Return the gradients of student_logits
    return student_logits.grad

#### Let's compute some gradients and visualize

In [ ]:
temperatures = [1.0, 2.0, 4.0, 8.0]
gradients = []
for T in temperatures:
    # Clone logits to ensure fresh gradient computation for each iteration
    student_logits_clone = student_logits.clone().detach()
    teacher_logits_clone = teacher_logits.clone().detach()

    grad = compute_kl_gradient(student_logits_clone, teacher_logits_clone, T)
    gradients.append(torch.norm(grad))

In [ ]:
#@title Visualize
# Convert gradient norms to numpy for plotting if they are tensors
plot_gradient_norms = [gn.item() for gn in gradients]

fig, ax = plt.subplots(figsize=(8, 5))

# Plot gradient norm vs temperature
ax.plot(temperatures, plot_gradient_norms, marker='o', linestyle='-', color='b', label='Observed Gradient Norm')

ax.set_xlabel('Temperature (T)')
ax.set_ylabel('Value')
ax.set_title('Gradient Norm vs. Temperature')
ax.legend()
ax.grid(True, linestyle=':', alpha=0.7)
plt.tight_layout()
plt.show()

### Scales for gradients

In [ ]:
# TODO: Change these four numbers to make sure the gradients don't
# change too much with temperatures
scales_for_gradients = [1, 1, 1, 1]

#### Solution

In [ ]:
# @title
scales_for_gradients = [t**2 for t in temperatures]

In [ ]:
#@title Visualise Scaled Gradients
# Convert gradient norms to numpy for plotting if they are tensors
plot_gradient_norms = [gn.item() for gn in gradients]

fig, ax = plt.subplots(figsize=(8, 5))

# Plot gradient norm vs temperature
ax.plot(temperatures, plot_gradient_norms, marker='o', linestyle='-', color='b', label='Observed Gradient Norm')

# Plot scaled_gradients
t_values_for_plot = np.array(temperatures)
inv_t_squared = np.asarray(plot_gradient_norms) * np.asarray(scales_for_gradients)
ax.plot(temperatures, inv_t_squared, linestyle='--', color='r', label='Scaled Gradient Norm')

ax.set_xlabel('Temperature (T)')
ax.set_ylabel('Value')
ax.set_title('Gradient Norm vs. Temperature with scaling')
ax.legend()
ax.grid(True, linestyle=':', alpha=0.7)
plt.tight_layout()
plt.show()

As the temperature T increases, the gradient of the loss gets rescaled by a factor of (1/T)^2. This has been derived in the original Hinton paper. As decreasing gradient value can affect optimization dynamics, we need to upscale the gradients by a factor of T^2. The easiest way to do it is just to multiply the loss by T^2.

Note: we are talking about distillation for classification, but it can be applied to other tasks. This temperature-scaling result holds only for classification.

### Activity 6. Implement the `TemperedDistillationLoss` class.
It should be similar to `DistillationLoss` but with the temperature scaling applied to the teacher and student probabilities. The loss should be rescaled by temperature.



In [ ]:
class TemperedDistillationLoss(nn.Module):
    """Knowledge Distillation Loss with Temperature Scaling.

    Combine the hard loss computed on labels with the soft teacher loss. Use
    Alpha weight on the hard loss and (1 - alpha) on the soft loss.
    """
    def __init__(self, temperature: float, alpha: float) -> None:
        """Initialize TemperedDistillationLoss.

        Args:
            temperature (float): Temperature for softening the distribution.
            alpha (float): Weight for hard loss (1-alpha for soft loss).
        """
        super().__init__()
        # TODO

    def forward(
        self,
        student_logits: torch.Tensor,
        teacher_logits: torch.Tensor,
        labels: torch.Tensor
    ) -> Tuple[torch.Tensor, torch.Tensor, torch.Tensor]:
        """Compute the tempered distillation loss.

        Args:
            student_logits: Logits from student model.
            teacher_logits: Logits from teacher model.
            labels: True labels.

        Returns:
            tuple: (total_loss, hard_loss, soft_loss)
        """
        # TODO
        metrics = dict(hard_loss=0., soft_loss=0)
        return total_loss, metrics

#### Solution

In [ ]:
# @title
class TemperedDistillationLoss(nn.Module):
    """Knowledge Distillation Loss with Temperature Scaling.

    Combines:
    1. Hard label loss: CrossEntropy(student_output, true_labels)
    2. Soft label loss: KLDiv(student_soft, teacher_soft) * T^2

    The T^2 factor compensates for the gradient magnitude reduction when using
    temperature.
    """
    def __init__(self, temperature: float = 4.0, alpha: float = 0.3) -> None:
        """Initialize TemperedDistillationLoss.

        Args:
            temperature (float): Temperature for softening the distribution.
            alpha (float): Weight for hard loss (1-alpha for soft loss).
        """
        super().__init__()
        self.temperature = temperature
        self.alpha = alpha
        self.ce_loss = nn.CrossEntropyLoss()
        self.kl_loss = nn.KLDivLoss(reduction='batchmean')

    def forward(
        self,
        student_logits: torch.Tensor,
        teacher_logits: torch.Tensor,
        labels: torch.Tensor
    ) -> Tuple[torch.Tensor, torch.Tensor, torch.Tensor]:
        """Compute the tempered distillation loss.

        Args:
            student_logits: Logits from student model.
            teacher_logits: Logits from teacher model.
            labels: True labels.

        Returns:
            tuple: (total_loss, hard_loss, soft_loss)
        """
        # Hard label loss (standard cross-entropy)
        hard_loss = self.ce_loss(student_logits, labels)

        # Soft label loss (KL divergence with temperature)
        # Student: log-softmax with temperature
        student_soft = F.log_softmax(student_logits / self.temperature, dim=1)
        # Teacher: softmax with temperature (target distribution)
        teacher_soft = F.softmax(teacher_logits / self.temperature, dim=1)

        # KL divergence * T^2 (to match gradient magnitude)
        soft_loss = self.kl_loss(student_soft, teacher_soft) * (self.temperature ** 2)

        # Combined loss
        total_loss = self.alpha * hard_loss + (1 - self.alpha) * soft_loss

        metrics = dict(hard_loss=hard_loss, soft_loss=soft_loss)
        return total_loss, metrics

### Training with temperature-scaled distillation (Using big network as teacher)

In [ ]:
# Distillation configuration
ALPHA = 0.1        # Weight for hard loss (try: 0.1, 0.3, 0.5)
TEMPERATURE = 4.0  # Try values in [1, 20]
LR = 0.01


# Load trained teacher
teacher.load_state_dict(torch.load('teacher_resnet18.pth'))
teacher.eval()

teacher_dim = teacher.fc.in_features
student_tempered = StudentCNN(output_dim=teacher_dim).to(device)

# Setup training
tempered_distill_criterion = TemperedDistillationLoss(
    temperature=TEMPERATURE,
    alpha=ALPHA
)

tempered_distillation_history = train_loop(
    model=student_tempered,
    teacher=teacher,
    criterion=tempered_distill_criterion,
    train_data_loader=trainloader,
    test_data_loader=testloader,
    num_epochs=STUDENT_EPOCHS,
    learning_rate=LR,
    checkpoint_name="student_distilled_tempered_v1",
    model_name="Tempered-distillation Student",
    experiment_name="Training Student with Tempered Knowledge Distillation",
    overwrite=True,
)

In [ ]:
#@title Plot training curves
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Test accuracy comparison (axes[0] remains as is, it already has distinct labels and lines)
xs = np.arange(1, len(baseline_history['test_acc']) + 1)
axes[0].plot(xs, baseline_history['test_acc'], label='Student (Baseline)', linestyle='--')
axes[0].plot(xs, distilled_history['test_acc'], label='Student (Distilled)', linewidth=2)
axes[0].plot(xs, tempered_distillation_history['test_acc'], label='Student (Tempered Distillation)', linewidth=2)
axes[0].axhline(y=teacher_acc, color='r', linestyle=':', label=f'Teacher ({teacher_acc:.1f}%)')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Test Accuracy (%)')
axes[0].set_title('Test Accuracy Comparison')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Loss breakdown for distilled student (axes[1])
# Define consistent colors for the loss components
hard_loss_color = 'tab:blue'
soft_loss_color = 'tab:orange'
total_loss_color = 'tab:green'

# Plot original distillation losses with solid lines
axes[1].plot(xs, to_cpu(distilled_history['hard_loss']), label='Hard Loss (Distilled)', alpha=0.8, color=hard_loss_color)
axes[1].plot(xs, to_cpu(distilled_history['soft_loss']), label='Soft Loss (Distilled)', alpha=0.8, color=soft_loss_color)
axes[1].plot(xs, to_cpu(distilled_history['loss']), label='Total Loss (Distilled)', linewidth=2, color=total_loss_color)

# Plot tempered distillation losses with dashed lines and the same colors
axes[1].plot(xs, to_cpu(tempered_distillation_history['hard_loss']), alpha=0.8, color=hard_loss_color, linestyle='--')
axes[1].plot(xs, to_cpu(tempered_distillation_history['soft_loss']), alpha=0.8, color=soft_loss_color, linestyle='--')
axes[1].plot(xs, to_cpu(tempered_distillation_history['loss']), linewidth=2, color=total_loss_color, linestyle='--')
axes[1].plot([], [], linewidth=2, linestyle='--', c='k', label='Tempered')

axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Loss')
axes[1].set_title('Distillation Loss Components')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 13. Distillation Aids Generalization
Sometimes you may have a large model pre-trained on a large dataset A, but you may want to train a different model on a different, often smaller, dataset B. If the two datasets are related, it may be possible to transfer knowledge from one to the other.

To do that, you take the teacher model trained on the dataset A and distill it into the student trained on the dataset B. In this case, the teacher is evaluated on the B dataset during distillation.

To simulate this scenario, we will take a teacher trained on Cifar-10 (our dataset A) and distill it into a student trained on a modified version of Cifar-19 (dataset B).

Let's start with modifying the Cifar-10 dataset.

In [ ]:
# Conver Cifar-10 to grayscale and add noise (old camera simulation)
class AddGaussianNoise:
    """Add Gaussian noise to a tensor."""
    def __init__(self, mean: float = 0., std: float = 0.1) -> None:
        """Initialize the transform.

        Args:
            mean (float): Mean of the noise.
            std (float): Standard deviation of the noise.
        """
        self.mean = mean
        self.std = std

    def __call__(self, tensor: torch.Tensor) -> torch.Tensor:
        """Apply noise to the tensor.

        Args:
            tensor (torch.Tensor): Input tensor.

        Returns:
            torch.Tensor: Noisy tensor.
        """
        noise = torch.randn_like(tensor) * self.std + self.mean
        return torch.clamp(tensor + noise, 0., 1.)

transform_train_grayscale_noisy = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.Grayscale(num_output_channels=3),
    transforms.ToTensor(),
    AddGaussianNoise(mean=0., std=0.02),
    transforms.Normalize(cifar_mean, cifar_std),
])

transform_test_grayscale_noisy = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),
    transforms.ToTensor(),
    AddGaussianNoise(mean=0., std=0.02),
    transforms.Normalize(cifar_mean, cifar_std),
])

# Download and load datasets
augmented_cifar10_train = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train_grayscale_noisy)
augmented_cifar10_test = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test_grayscale_noisy)

augmented_cifar10_trainloader = DataLoader(augmented_cifar10_train, batch_size=128, shuffle=True, num_workers=0)
augmented_cifar10_testloader = DataLoader(augmented_cifar10_test, batch_size=128, shuffle=False, num_workers=0)

### Activity 7. Plot some instances
Similar to the original CIFAR-10 dataset, plot a few sample instances to assess the impact of the applied transformation.

In [ ]:
# Get some random training images
dataiter_v2 = iter(augmented_cifar10_trainloader)
images, labels = next(dataiter_v2)

# Show images
# TODO

#### Solution

In [ ]:
# @title
# Get some random training images
dataiter_v2 = iter(augmented_cifar10_trainloader)
images, labels = next(dataiter_v2)

# Show images
fig, axes = plt.subplots(1, 8, figsize=(6, 1))
for i in range(8):
    imshow(axes[i], images[i])
    axes[i].set_title(classes[labels[i]])
    axes[i].axis('off')
plt.tight_layout()
plt.show()

### Activity 8. Train the student without distillation on the new dataset.

In [ ]:
# TODO

#### Solution

In [ ]:
# @title
# Train student from scratch (no teacher)
STUDENT_EPOCHS = 5
LR = 0.01

teacher_dim = teacher.fc.in_features
aug_cifar10_student_baseline = StudentCNN(output_dim=teacher_dim).to(device)

aug_cifar10_student_baseline_history = train_loop(
    model=aug_cifar10_student_baseline,
    teacher=None,
    criterion=nn.CrossEntropyLoss(),
    train_data_loader=augmented_cifar10_trainloader,
    test_data_loader=augmented_cifar10_testloader,
    num_epochs=STUDENT_EPOCHS,
    learning_rate=LR,
    checkpoint_name="augmented_cifar_student_no_distillation",
    model_name="Augmented Cifar-10 Baseline",
    experiment_name="Training Student Baseline (NO distillation)",
    overwrite=True,
)

##### Download the checkpoint

In [ ]:
!gdown --id 19vQMX9RdA7LRq6gssRL8FVlapcv0KNRx

aug_cifar10_student_baseline = StudentCNN(output_dim=teacher_dim).to(device)
aug_cifar10_student_baseline.load_state_dict(torch.load('augmented_cifar_student_no_distillation.pth'))

### Activity 9. Train the student network on the new dataset with distillation
We will use the TemperedDistillationLoss class, and compare the results against those obtained from training the student model independently.

In [ ]:
# TODO

#### Solution

In [ ]:
# @title
# Distillation configuration
STUDENT_EPOCHS = 5
TEMPERATURE = 4.0  # Try: 2, 4, 8, 20
ALPHA = 0.1        # Weight for hard loss (try: 0.1, 0.3, 0.5)
LR = 0.01

# Load trained teacher
teacher.load_state_dict(torch.load('teacher_resnet18.pth'))
teacher.eval()

teacher_dim = teacher.fc.in_features
aug_cifar10_student_distilled = StudentCNN(output_dim=teacher_dim).to(device)
distill_criterion = TemperedDistillationLoss(temperature=TEMPERATURE, alpha=ALPHA)

aug_cifar10_student_distilled_history = train_loop(
    model=aug_cifar10_student_distilled,
    teacher=teacher,
    criterion=distill_criterion,
    train_data_loader=augmented_cifar10_trainloader,
    test_data_loader=augmented_cifar10_testloader,
    num_epochs=STUDENT_EPOCHS,
    learning_rate=LR,
    checkpoint_name="augmented_cifar_distilled_student",
    model_name="Augmented Cifar-10 Distilled Student",
    experiment_name="Training Distilled student on augmented Cifar-10",
    overwrite=True,
)

In [ ]:
#@title Plot training curves
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

xs = np.arange(1, STUDENT_EPOCHS + 1)
# Test accuracy comparison
axes[0].plot(xs, aug_cifar10_student_baseline_history['test_acc'], label='Student (Baseline)', linestyle='--')
axes[0].plot(xs, aug_cifar10_student_distilled_history['test_acc'], label='Student (Distilled)', linewidth=2)
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Test Accuracy (%)')
axes[0].set_title('Test Accuracy Comparison')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

hard_loss_col = "tab:blue"
# Loss breakdown for distilled student
axes[1].plot(xs, to_cpu(aug_cifar10_student_distilled_history['hard_loss']), label='Hard Loss (CE)', alpha=0.8)
axes[1].plot(xs, to_cpu(aug_cifar10_student_distilled_history['soft_loss']), label='Soft Loss (KL)', alpha=0.8)
axes[1].plot(xs, to_cpu(aug_cifar10_student_distilled_history['loss']), label='Total Loss', linewidth=2)
axes[1].plot(
    xs, aug_cifar10_student_baseline_history['loss'],
    label='Hard Loss (Baseline)', alpha=0.8, c=hard_loss_col, linestyle='--'
)
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Loss')
axes[1].set_title('Distillation Loss Components')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 14. Multi-Expert Distillation

To improve model performance, you can train several neural networks and use them together as an ensemble. A prediction from an ensemble $$x_\text{ensemble} = \frac{1}{n}\sum_{i=1}^{n}x_i$$ is an average of the individual model predictions $x_i$.

An ensemble is usually stronger than each of the individual models. It turns out that we can distill not just one model but the whole ensemble into the student network. That's what we will do now.

In [ ]:
teacher2 = TeacherCNN().to(device)

In [ ]:
# Download the second teacher ckpt
!gdown --id 1P7wadGFV7ZPdJsXd2B91nKEEnQOst00G

teacher2.load_state_dict(torch.load('teacher_resnet18_v2.pth'))
teacher2.eval()

### Alternative: Train a new teacher

In [ ]:
# @title
# Set a different seed and train a new teacher
set_seed(20)

# Training configuration
TEACHER_EPOCHS = 15  # Increase for better teacher (50+ recommended)
LEARNING_RATE = 0.01

teacher2_history = train_loop(
    model=teacher2,
    criterion=nn.CrossEntropyLoss(),
    train_data_loader=trainloader,
    test_data_loader=testloader,
    num_epochs=TEACHER_EPOCHS,
    learning_rate=LEARNING_RATE,
    checkpoint_name="teacher_resnet18_v2.pth",
    model_name="Teacher",
    experiment_name="Teacher Model (ResNet-18)",
    overwrite=False,

)

set_seed(10)

### Activity 10. Modify `train_epoch` to accept a list of teachers.
For each teacher, compute the soft loss, then average them to get the final distillation loss.

In [ ]:
def train_epoch_multi_teacher(model, teacher, trainloader, criterion, optimizer, device):
    # TODO
    pass

### Solution

In [ ]:
# @title
def train_epoch_multi_teacher(model, teacher, trainloader, criterion, optimizer, device):
    """Trains the model using supervised training or distillation.

    Distillation is used only when the teacher is present. In that case, the
    teacher is frozen (no gradients).
    """
    model.train()


    running_loss = 0.0
    running_hard_loss = 0.0
    running_soft_loss = 0.0
    correct = 0
    total = 0
    running_metrics = collections.defaultdict(int)

    for inputs, labels in tqdm.tqdm(trainloader, desc="Training", leave=False):
        inputs, labels = inputs.to(device), labels.to(device)

        # Forward pass for the trained model
        optimizer.zero_grad()
        logits = model(inputs)

        # Compute loss
        if teacher is not None:
            # Get teacher predictions (no gradient needed)
            if isinstance(teacher, list):
                avg_soft_loss = 0.0
                for t_model in teacher:
                    t_model.eval()
                    with torch.no_grad():
                        teacher_logits = t_model(inputs)
                    _, metrics = criterion(
                        logits, teacher_logits, labels
                    )
                    avg_soft_loss += metrics.pop('soft_loss')
                avg_soft_loss /= len(teacher)
                metrics['soft_loss'] = avg_soft_loss
                # hard loss is the same for every teacher
                loss = criterion.alpha * metrics['hard_loss'] + (1 - criterion.alpha) * avg_soft_loss
            else:
                teacher.eval()  # Teacher is always in eval mode
                with torch.no_grad():
                    teacher_logits = teacher(inputs)
                loss, metrics = criterion(
                    logits, teacher_logits, labels
                )
        else:
          loss = criterion(logits, labels)
          metrics = {}

        # Backward and optimize
        loss.backward()
        optimizer.step()

        # Metrics
        running_metrics['loss'] += loss.item()
        for k, v in metrics.items():
            running_metrics[k] += v.item()
        _, predicted = logits.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    n = len(trainloader)
    results = {k: v/n for k, v in running_metrics.items()}
    results['train_acc'] = 100. * correct / total
    return results

### Activity 11. Train the student using the 2 experts

In [ ]:
# TODO
multi_teacher_history = None

### Solution

In [ ]:
# @title
# Distillation configuration
STUDENT_EPOCHS = 5
ALPHA = 0.1        # Weight for hard loss (try: 0.1, 0.3, 0.5)
TEMPERATURE = 4.0  # Try values in [1, 20]
LR = 0.01


# Load trained teacher
teacher1 = TeacherCNN().to(device)
teacher1.load_state_dict(torch.load('teacher_resnet18.pth'))
teacher1.eval()
teacher2 = TeacherCNN().to(device)
teacher2.load_state_dict(torch.load('teacher_resnet18_v2.pth'))
teacher2.eval()

teacher_dim = teacher.fc.in_features
student_multi_distilled = StudentCNN(output_dim=teacher_dim).to(device)

# Setup training
tempered_distill_criterion = TemperedDistillationLoss(
    temperature=TEMPERATURE,
    alpha=ALPHA
)

multi_teacher_history = train_loop(
    model=student_multi_distilled,
    teacher=[teacher1, teacher2],
    criterion=tempered_distill_criterion,
    train_data_loader=trainloader,
    test_data_loader=testloader,
    num_epochs=STUDENT_EPOCHS,
    learning_rate=LR,
    checkpoint_name="student_multi_distilled",
    model_name="Multi-Teacher Student",
    experiment_name="Training Student with Tempered Multi-Expert Distillation",
    overwrite=True,
    # Use a different train_epoch_function
    custom_train_epoch_fn=train_epoch_multi_teacher,
)

In [ ]:
#@title Plot training curves
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

xs = np.arange(1, STUDENT_EPOCHS + 1)
# Test accuracy comparison
axes[0].plot(xs, tempered_distillation_history['test_acc'], label='Single-Teacher Student', linestyle='--')
axes[0].plot(xs, multi_teacher_history['test_acc'], label='Multi-Teacher Student', linewidth=2)
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Test Accuracy (%)')
axes[0].set_title('Test Accuracy Comparison')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

hard_loss_color = 'tab:blue'
soft_loss_color = 'tab:orange'
total_loss_color = 'tab:green'
# Loss breakdown for distilled student
axes[1].plot(xs, to_cpu(multi_teacher_history['hard_loss']), label='Hard Loss (CE)', alpha=0.8, c=hard_loss_color)
axes[1].plot(xs, to_cpu(multi_teacher_history['soft_loss']), label='Soft Loss (KL)', alpha=0.8, c=soft_loss_color)
axes[1].plot(xs, to_cpu(multi_teacher_history['loss']), label='Total Loss', linewidth=2, c=total_loss_color)
axes[1].plot(xs, to_cpu(tempered_distillation_history['hard_loss']), linestyle='--', alpha=0.8, c=hard_loss_color)
axes[1].plot(xs, to_cpu(tempered_distillation_history['soft_loss']), linestyle='--', alpha=0.8, c=soft_loss_color)
axes[1].plot(xs, to_cpu(tempered_distillation_history['loss']), linestyle='--', linewidth=2, c=total_loss_color)
axes[1].plot([], [], linestyle='--', linewidth=2, c='k', label='Single Teacher')


axes[1].plot(
    xs, aug_cifar10_student_baseline_history['loss'],
    label='Hard Loss (Baseline)', alpha=0.8, c=hard_loss_col, linestyle='--'
)


axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Loss')
axes[1].set_title('Distillation Loss Components')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 15. Feature Distillation
In addition to distilling features, we'll be looking at distilling intermediate features produced by the neural network.

This technique allows more efficient transfer of the "hidden" knowledge, but it is more complicated. As the teacher may have a different architecture than the student, we need to match a teacher layer that we want to distill from with a student layer that we want to distill to. These layers may have different dimensionalities, so we might need to adapter the student features. We can use a linear layer to do so.

The loss used for feature distilation is usually based on cosine similarity. Since we want something that gets better as it gets smaller, we take 1 - cosine similarity.

### Activity 12: Implement feature distillation loss

In [ ]:
class FeatureDistillationLoss(nn.Module):
    """
    Feature-based distillation: match intermediate representations.
    Requires a projection layer if dimensions don't match.
    """
    def __init__(self, device, student_dim, teacher_dim, temperature=4.0, alpha=0.3, beta=0.5):
        super().__init__()
        self.temperature = temperature
        self.alpha = alpha
        self.beta = beta  # Weight for feature loss

        # Projection layer if dimensions don't match
        self.projector = nn.Linear(student_dim, teacher_dim) if student_dim != teacher_dim else nn.Identity()
        self.projector = self.projector.to(device)

        self.ce_loss = nn.CrossEntropyLoss()
        self.kl_loss = nn.KLDivLoss(reduction='batchmean')

    def forward(self, student_logits, teacher_logits, student_features, teacher_features, labels):
        # TODO implement tempered-distillation loss we used before and combine
        # it with a new feature distillation loss. Read the above section for
        # how to implement it.


        metrics = dict(
            hard_loss=0.,
            soft_loss=0.,
            feature_loss=0.
        )
        return loss, metrics

#### Solution

In [ ]:
# @title
class FeatureDistillationLoss(nn.Module):
    """
    Feature-based distillation: match intermediate representations.
    Requires a projection layer if dimensions don't match.
    """
    def __init__(self, device, student_dim, teacher_dim, temperature=4.0, alpha=0.3, beta=0.5):
        super().__init__()
        self.temperature = temperature
        self.alpha = alpha
        self.beta = beta  # Weight for feature loss

        # Projection layer if dimensions don't match
        self.projector = nn.Linear(student_dim, teacher_dim) if student_dim != teacher_dim else nn.Identity()
        self.projector = self.projector.to(device)

        self.ce_loss = nn.CrossEntropyLoss()
        self.kl_loss = nn.KLDivLoss(reduction='batchmean')

    def forward(self, student_logits, teacher_logits, student_features, teacher_features, labels):
        # Standard distillation losses
        hard_loss = self.ce_loss(student_logits, labels)
        soft_loss = self.kl_loss(
            F.log_softmax(student_logits / self.temperature, dim=1),
            F.softmax(teacher_logits / self.temperature, dim=1)
        ) * (self.temperature ** 2)

        # Feature matching loss
        student_proj = self.projector(student_features)
        feature_loss = 1 - F.cosine_similarity(student_proj, teacher_features).mean()

        # Combined loss
        loss = self.alpha * hard_loss + (1 - self.alpha) * soft_loss + self.beta * feature_loss

        metrics = dict(
            hard_loss=hard_loss,
            soft_loss=soft_loss,
            feature_loss=feature_loss
        )
        return loss, metrics

### Train with Feature Distillation
As the signature of the loss function is different than before, we need to implement a different epoch-training function and a different training loop.

In [ ]:
def train_with_feature_distillation(student, teacher, trainloader, criterion, optimizer, device):
    """
    Train student using knowledge distillation from teacher.
    Teacher is frozen (no gradients).
    """
    student.train()
    teacher.eval()  # Teacher is always in eval mode

    running_metrics = collections.defaultdict(int)
    correct = 0
    total = 0

    for inputs, labels in tqdm.tqdm(trainloader, desc="Distilling", leave=False):
        inputs, labels = inputs.to(device), labels.to(device)

        # Get teacher predictions (no gradient needed)
        with torch.no_grad():
            teacher_logits, teacher_features = teacher(inputs, output_features=True)

        # Forward pass for student
        optimizer.zero_grad()
        student_logits, student_features = student(inputs, output_features=True)

        # Compute distillation loss
        loss, metrics = criterion(student_logits, teacher_logits, student_features, teacher_features, labels)

        # Backward and optimize
        loss.backward()
        optimizer.step()

        # Metrics
        running_metrics['loss'] += loss.item()
        for k, v in metrics.items():
            running_metrics[k] += v
        _, predicted = student_logits.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    n = len(trainloader)
    results = {k: v/n for k, v in running_metrics.items()}
    results['train_acc'] = 100. * correct / total
    return results

In [ ]:
# Distillation configuration
STUDENT_EPOCHS = 5
TEMPERATURE = 4.0  # Try: 2, 4, 8, 20
ALPHA = 0.1        # Weight for hard loss (try: 0.1, 0.3, 0.5)
LR = 0.01
BETA = 0.5

# Load trained teacher
teacher.load_state_dict(torch.load('teacher_resnet18.pth'))
teacher.eval()

# Create fresh student
teacher_dim = teacher.fc.in_features
student_feature_distilled = StudentCNN(output_dim=teacher_dim).to(device)

# Setup training
student_dim = student_feature_distilled.fc2[1].in_features
teacher_dim = teacher.fc.in_features
feature_distill_criterion = FeatureDistillationLoss(
    device, student_dim, teacher_dim,
    temperature=TEMPERATURE, alpha=ALPHA, beta=BETA
)
parameters = (
    list(student_feature_distilled.parameters())
     + list(feature_distill_criterion.projector.parameters())
)
# Setup optimizer and scheduler
optimizer = optim.Adam(parameters, lr=LR)
scheduler = optim.lr_scheduler.CosineAnnealingLR(
    optimizer, T_max=STUDENT_EPOCHS, eta_min=0.001
)

print(f"Training Student with Knowledge Distillation")
print(f"Temperature: {TEMPERATURE}, Alpha: {ALPHA}, Beta: {BETA}")
print("=" * 60)

history = collections.defaultdict(list)
for epoch in range(STUDENT_EPOCHS):
    train_start = time.time()

    # Run training epoch
    metrics = train_with_feature_distillation(
      student_feature_distilled, teacher, trainloader, feature_distill_criterion, optimizer, device
  )
    train_time = time.time() - train_start

    # Evaluation
    eval_start = time.time()
    metrics.update(evaluate(student_feature_distilled, testloader, device))
    eval_time = time.time() - eval_start

    # Update learning rate
    scheduler.step()

    # Store metrics
    for k, v in metrics.items():
        history[k].append(v)

    train_acc = metrics.pop('train_acc')
    test_acc = metrics.pop('test_acc')

    # Log progress
    print(
        f"Epoch {epoch+1:2d}/{STUDENT_EPOCHS}",
        ' |'.join(f" {k}: {v:.4f}" for k, v in metrics.items()),
        f"| Train Acc {train_acc:.2f}% | Test Acc {test_acc:.2f}%"
        f"| Epoch time {train_time:.2f}s | Eval time {eval_time:.2f}s"
    )

print(f"\n✅ Distilled Student final test accuracy: {test_acc:.2f}%")

## 16. 🧪 Experiment: Hyperparameter Sensitivity

Try different values of **Temperature** and **Alpha** to see their effects!

In [ ]:
def quick_distillation_experiment(temperature, alpha, epochs=15):
    """Quick experiment with different hyperparameters"""
    student = StudentCNN().to(device)
    criterion = DistillationLoss(temperature=temperature, alpha=alpha)
    optimizer = optim.Adam(student.parameters(), lr=0.001)

    for epoch in range(epochs):
        train_with_distillation(student, teacher, trainloader, criterion, optimizer, device)

    return evaluate(student, testloader, device)

# Uncomment to run experiments (takes a few minutes)
# print("Running hyperparameter experiments...")

# temperatures = [1, 2, 4, 8, 16]
# alphas = [0.1, 0.3, 0.5, 0.7]

# results = {}
# for T in temperatures:
#     for a in alphas:
#         acc = quick_distillation_experiment(T, a)
#         results[(T, a)] = acc
#         print(f"T={T}, α={a}: {acc:.2f}%")

## 17. Types of Knowledge Distillation

What we implemented is **Response-based Distillation**. There are other types:

| Type | What's Transferred | Example |
|------|-------------------|--------|
| **Response-based** | Final output logits | What we did! |
| **Feature-based** | Intermediate representations | FitNets, Attention Transfer |
| **Relation-based** | Relationships between samples | Contrastive distillation |

## 18. Bonus: Born Again Networks (Use the previous trained model (Student) as a Teacher).

You train a model once (the Teacher). Then, you take a fresh version of the exact same architecture (the Student) and train it to not only hit the correct labels but also to mimic the "confidence" of the Teacher. Surprisingly, the Student almost always ends up smarter than the Teacher. This concept was introduced by [Furlanello et al](https://arxiv.org/pdf/1805.04770).

In [ ]:
# Distillation configuration
STUDENT_EPOCHS = 5
TEMPERATURE = 4.0  # Try: 2, 4, 8, 20
ALPHA = 0.1        # Weight for hard loss (try: 0.1, 0.3, 0.5)
LR = 0.01

# Load trained teacher
teacher_dim = teacher.fc.in_features
teacher_st_baseline = StudentCNN(output_dim=teacher_dim).to(device)
teacher_st_baseline.load_state_dict(torch.load('baseline_student_v1.pth'))
teacher_st_baseline.eval()

ban_student_distilled = StudentCNN(output_dim=teacher_dim).to(device)

# Setup training
distill_criterion = TemperedDistillationLoss(temperature=TEMPERATURE, alpha=ALPHA)

ban_history = train_loop(
    student=ban_student_distilled,
    teacher=teacher_st_baseline,
    criterion=tempered_distill_criterion,
    train_data_loader=trainloader,
    test_data_loader=testloader,
    num_epochs=STUDENT_EPOCHS,
    learning_rate=LR,
    checkpoint_name="ban_v1",
    model_name="Born-Again Network",
    experiment_name="Training Student Born-Again Network",
    overwrite=False,
)

## 19. Key Takeaways

### ✅ What We Learned

1. **Knowledge Distillation** transfers "dark knowledge" from a large teacher to a small student

2. **Soft labels** contain richer information than hard labels (class relationships)

3. **Temperature** controls how soft the probability distribution is:
   - Higher T = more information transfer, but potentially noisier
   - Typical values: 2-20

4. **Alpha** balances hard and soft losses:
   - Lower α = more emphasis on mimicking teacher
   - Higher α = more emphasis on ground truth

5. **Results**: Distilled students typically outperform students trained from scratch by 1-5%

### 🚀 Extensions to Explore

- **Self-distillation**: Use the same architecture for teacher and student
- **Online distillation**: Train teacher and student simultaneously
- **Multi-teacher distillation**: Ensemble of teachers
- **Task-specific distillation**: For NLP, use DistilBERT approach

### 📚 References

1. Hinton et al., "Distilling the Knowledge in a Neural Network" (2015)
2. Romero et al., "FitNets: Hints for Thin Deep Nets" (2015)
3. Gou et al., "Knowledge Distillation: A Survey" (2021)

## 20. 💪 Exercise for You!

Try these modifications and see what happens:

1. **Change the student architecture**: Make it deeper or wider
2. **Try different temperatures**: Plot accuracy vs temperature

In [ ]:
# Your experiments here!
#
# Example: Try a deeper student
# class DeeperStudent(nn.Module):
#     def __init__(self):
#         super().__init__()
#         # Your architecture here
#         pass

Implement computing gradient norms and compare the student network with and without distillation, and with different temperatures during distillation

In [ ]:
grad_norm = torch.sqrt(sum([torch.norm(p.grad)**2 for p in student_baseline.parameters()]))
print(f'{grad_norm=}')
# TODO: Do this across model training

Compare the norm of model parameters for students trained with only real data and the ones trained with different forms of distillation. What do you see? How can you explain that?

In [ ]:
param_norm = torch.sqrt(sum([torch.norm(p.data)**2 for p in student_baseline.parameters()]))
print(f'{param_norm=}')